# Reinforcement Learning
***Eric Gu <br>
December 3, 2018***

- - -

This notebook will explore two interesting Markov Decision Process (MDP) problems:
* [Frozen Lake](https://gym.openai.com/envs/FrozenLake-v0/)
* [Taxi](https://gym.openai.com/envs/Taxi-v2/) [Dietterich2000]

We will explore them using two fundamental methods for solving MDPs:
* Value-iteration (VI)
* Policy-iteration (PI)

Additionally, we will then use a famous reinforcement learning algorithm to solve the same problems: 
* Q-learning

*Written in Jupyter Notebook using OpenAI's Gym [environments](https://gym.openai.com/envs/), and with algorithm implementation borrowed from [Moustafa Alzantot](https://medium.com/@m.alzantot/deep-reinforcement-learning-demysitifed-episode-2-policy-iteration-value-iteration-and-q-978f9e89ddaa), [Arthur Juliani](https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-0-q-learning-with-tables-and-neural-networks-d195264329d0), and [Denny Britz](https://github.com/dennybritz/reinforcement-learning/tree/master/DP).*

In [1]:
from timeit import default_timer as timer
import numpy as np
import gym
from gym import wrappers

## Description of Problems

### Frozen Lake
Taken from OpenAI's description of [FrozenLake-v0](https://gym.openai.com/envs/FrozenLake-v0/):
"The agent controls the movement of a character in a grid world. Some tiles of the grid are walkable, and others lead to the agent falling into the water. Additionally, the movement direction of the agent is uncertain and only partially depends on the chosen direction. The agent is rewarded for finding a walkable path to a goal tile." 

The surface is described using a grid like the following:
```
SFFF       (S: starting point, safe)
FHFH       (F: frozen surface, safe)
FFFH       (H: hole, fall to your doom)
HFFG       (G: goal, where the frisbee is located)
```
An episode ends when the agent either reaches the goal or falls in a hole. You receive a reward of 1 if you reach the goal, and zero otherwise.

Immediately, we should note that although the state space of this stochastic MDP is small (4x4=16), at each step, there is only a 1/3 chance going where you want to go, and a 1/3 chance you go to the left or right instead. This makes the map treacherous, since to successfully navigate to the goal, the agent has to thread a narrow path between 2 holes no matter what. On the upside, the problem is very simple because it has only one goal state. Let's see how our algorithms perform.

### Taxi
There are four designated locations in the grid world indicated by R(ed), B(lue), G(reen), and Y(ellow). When the episode starts, the taxi starts off at a random square and the passenger is at a random location. The taxi drives to the passenger's location, picks up the passenger, drives to the passenger's destination (another one of the four specified locations), and then drops off the passenger. Once the passenger is dropped off, the episode ends. 

The grid world looks like the following:
```
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
```

##### Actions
There are 6 discrete deterministic actions:
- 0: move south
- 1: move north
- 2: move east 
- 3: move west 
- 4: pickup passenger
- 5: dropoff passenger

##### Rewards
There is a reward of -1 for each action and an additional reward of +20 for delivering the passenger. There is a reward of -10 for executing actions "pickup" and "dropoff" illegally.

Although the grid world appears to be 5x5, there are actually 500 discrete states because the passenger can be in 5 possible locations (including in the taxi) and 4 destination locations.  The Taxi Problem is an example of a hierarchical MDP—a more complex problem involving subgoals and many permutations of destinations & passengers can be abstracted by encoding relevant subproblem information into the state. While this MDP is deterministic, it poses a challenge because of the far larger state space. 

## Frozen Lake
From what we know, we expect PI to converge in fewer iterations than VI, and in probably less wall clock time because of the greater computational requirements required by VI. 

### Value Iteration (VI)
We initialize the value functions to arbitrary random values. Let's start by running VI with a variety of values of gamma (discount rate):

In [8]:
"""
Solving FrozenLake environment using Value-Itertion.
"""

env_name  = 'FrozenLake-v0'
env = gym.make(env_name)
env = env.unwrapped

def FrozenLakeVI(g = 1.0, n = 100, render = False):
    gamma = g

    env.seed(0)
    np.random.seed(0)

    start = timer()
    optimal_v = value_iteration(env, gamma);
    end = timer()

    policy = extract_policy(optimal_v, gamma)
    policy_score = evaluate_policy(env, policy, gamma, n=n)

    print('Policy average score = ', policy_score)
    print('Time elapsed: ', end - start)

        
def run_episode(env, policy, gamma = 1.0, render = False):
    """ Evaluates policy by using it to run an episode and finding its
    total reward.
    args:
    env: gym environment.
    policy: the policy to be used.
    gamma: discount factor.
    render: boolean to turn rendering on/off.
    returns:
    total reward: real value of the total reward recieved by agent under policy.
    """
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        obs, reward, done , _ = env.step(int(policy[obs]))
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward


def evaluate_policy(env, policy, gamma = 1.0,  n = 100):
    """ Evaluates a policy by running it n times.
    returns:
    average total reward
    """
    scores = [
            run_episode(env, policy, gamma = gamma, render = False)
            for _ in range(n)]
    return np.mean(scores)

def extract_policy(v, gamma = 1.0):
    """ Extract the policy given a value-function """
    policy = np.zeros(env.nS)
    for s in range(env.nS):
        q_sa = np.zeros(env.action_space.n)
        for a in range(env.action_space.n):
            for next_sr in env.P[s][a]:
                # next_sr is a tuple of (probability, next state, reward, done)
                p, s_, r, _ = next_sr
                q_sa[a] += (p * (r + gamma * v[s_]))
        policy[s] = np.argmax(q_sa)
    return policy


def value_iteration(env, gamma = 1.0):
    """ Value-iteration algorithm """
    v = np.zeros(env.nS)  # initialize value-function
    max_iterations = 100000
    eps = 1e-20
    for i in range(max_iterations):
        prev_v = np.copy(v)
        for s in range(env.nS):
            q_sa = [sum([p*(r + prev_v[s_]) for p, s_, r, _ in env.P[s][a]]) for a in range(env.nA)] 
            v[s] = max(q_sa)
        if (np.sum(np.fabs(prev_v - v)) <= eps):
            print ('Value-iteration converged at iteration# %d.' %(i+1))
            break
    return v

In [20]:
for gamma in [0.1, 0.5, 0.9, 0.95, 0.99, 0.999, 1.0]:
    FrozenLakeVI(g=gamma)

Value-iteration converged at iteration# 1373.
Policy average score =  1.0100312304110134e-07
Time elapsed:  0.19633457733834803
Value-iteration converged at iteration# 1373.
Policy average score =  0.00043486333365813445
Time elapsed:  0.19288121571753436
Value-iteration converged at iteration# 1373.
Policy average score =  0.07167804518900966
Time elapsed:  0.19060545382308192
Value-iteration converged at iteration# 1373.
Policy average score =  0.18807430252169385
Time elapsed:  0.1956689774933693
Value-iteration converged at iteration# 1373.
Policy average score =  0.5598022541207036
Time elapsed:  0.19080994347223168
Value-iteration converged at iteration# 1373.
Policy average score =  0.8184605786228931
Time elapsed:  0.19331967379730486
Value-iteration converged at iteration# 1373.
Policy average score =  0.86
Time elapsed:  0.19654558334968897


In [21]:
FrozenLakeVI(n = 1000)

Value-iteration converged at iteration# 1373.
Policy average score =  0.85
Time elapsed:  0.19847070290052216


Value iteration for Frozen Lake converges at iteration #1373. We can graph the average score for 100 episodes for each value of gamma and find that VI performs best at gamma = 1.0, which makes sense—there is only one goal state, with no other rewards or punishments that are closer or farther away. The policy extracted from the optimal values has an average score of 0.85 over 1000 episodes, and takes 0.198 seconds to run.

### Policy Iteration (PI)

The performance of PI against different values of gamma will be the same as VI's, so we use the same hyperparameters as in VI and compare the resulting scores.

In [23]:
"""
Solving FrozenLake environment using Policy iteration.
"""

env_name  = 'FrozenLake-v0'
env = gym.make(env_name)
env = env.unwrapped

def FrozenLakePI(g = 1.0, n = 100, render = False):
    g = g
    
    env.seed(0)
    np.random.seed(0)
    
    start = timer()
    optimal_policy = policy_iteration(env, gamma = g)
    end = timer()
    
    scores = evaluate_policy(env, optimal_policy, gamma = g, n = n)
    print('Average scores = ', np.mean(scores))
    print('Time elapsed: ', end - start)

def run_episode(env, policy, gamma = 1.0, render = False):
    """ Runs an episode and return the total reward """
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        obs, reward, done , _ = env.step(int(policy[obs]))
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward


def evaluate_policy(env, policy, gamma = 1.0, n = 100):
    scores = [run_episode(env, policy, gamma, False) for _ in range(n)]
    return np.mean(scores)

def extract_policy(v, gamma = 1.0):
    """ Extract the policy given a value-function """
    policy = np.zeros(env.nS)
    for s in range(env.nS):
        q_sa = np.zeros(env.nA)
        for a in range(env.nA):
            q_sa[a] = sum([p * (r + gamma * v[s_]) for p, s_, r, _ in  env.P[s][a]])
        policy[s] = np.argmax(q_sa)
    return policy

def compute_policy_v(env, policy, gamma=1.0):
    """ Iteratively evaluate the value-function under policy.
    Alternatively, we could formulate a set of linear equations in iterms of v[s] 
    and solve them to find the value function.
    """
    v = np.zeros(env.nS)
    eps = 1e-10
    while True:
        prev_v = np.copy(v)
        for s in range(env.nS):
            policy_a = policy[s]
            v[s] = sum([p * (r + gamma * prev_v[s_]) for p, s_, r, _ in env.P[s][policy_a]])
        if (np.sum((np.fabs(prev_v - v))) <= eps):
            # value converged
            break
    return v

def policy_iteration(env, gamma = 1.0):
    """ Policy-Iteration algorithm """
    policy = np.random.choice(env.nA, size=(env.nS))  # initialize a random policy
    max_iterations = 200000
    gamma = 1.0
    for i in range(max_iterations):
        old_policy_v = compute_policy_v(env, policy, gamma)
        new_policy = extract_policy(old_policy_v, gamma)
        if (np.all(policy == new_policy)):
            print ('Policy-Iteration converged at step %d.' %(i+1))
            break
        policy = new_policy
    return policy

In [25]:
FrozenLakePI(n = 1000)

Policy-Iteration converged at step 4.
Average scores =  0.85
Time elapsed:  0.09478203854814637


Not too surprising! PI converges in only 4 steps. When evaluating the optimal policy provided by the algorithm, PI has the same average score of 0.85 across 1000 episodes, but only requires 0.0948 seconds to run. Here, the principle advantage over VI is the speed at which the algorithm converges on a solution. PI seems to solve Frozen Lake about twice as quickly as VI. 

### Q-Learning

Given the small state space, we can use a simple table implementation of Q-Learning to store the values for how good it is to take a given action within a given state. In the case of the Frozen Lake environment, we have 4 possible actions (the four directions of movement) for each of 16 possible states, giving us a 16x4 table of Q-values. We start by initializing the table to be uniform (all zeros), and then as we observe the rewards we obtain for various actions, we update the table accordingly.

Instead of using an epsilon-greedy or "epsilon-decreasing" strategy, we adopt a slightly modified, more robust exploration strategy that adds noise to the greedy selection of the next action from the Q-table in proportion to the size of the Q-values, and which decreases in later iterations as time goes on ([source](https://medium.com/@awjuliani/hi-sung-4f446e95c9b5)). 

In [24]:
"""
Solving FrozenLake environment using Q-Learning
"""

env_name = 'FrozenLake-v0'
env = gym.make(env_name)
env = env.unwrapped
t_max = 10000

def FrozenLakeQLearn_NoisyQ(iter_max = 10000, learning_rate = 0.8, g = 1.0):
    # Hyperparameters
    num_episodes = iter_max
    # Set learning parameters
    lr = learning_rate
    y = g
    
    env.seed(0)
    np.random.seed(0)
    
    start = timer()
    
    #Initialize table with all zeros
    Q = np.zeros([env.observation_space.n,env.action_space.n])
    #create lists to contain total rewards and steps per episode
    #jList = []
    rList = []
    for i in range(num_episodes):
        #Reset environment and get first new observation
        s = env.reset()
        rAll = 0
        d = False
        j = 0
        #The Q-Table learning algorithm
        while j < 99:
            j+=1
            #Choose an action by greedily (with noise) picking from Q table
            a = np.argmax(Q[s,:] + np.random.randn(1,env.action_space.n)*(1./(i+1)))
            #Get new state and reward from environment
            s1,r,d,_ = env.step(a)
            #Update Q-Table with new knowledge
            Q[s,a] = Q[s,a] + lr*(r + y*np.max(Q[s1,:]) - Q[s,a])
            rAll += r
            s = s1
            if d == True:
                break
        #jList.append(j)
        rList.append(rAll)
    
    end = timer()
    
    print ("Score over time: " +  str(sum(rList)/num_episodes))
    print('Time elapsed: ', end - start)

In [13]:
for gamma in [0.1, 0.5, 0.9, 0.95, 0.99, 0.999, 1.0]:
    FrozenLakeQLearn(iter_max=1000, g=gamma)

Score over time: 0.019
Time elapsed:  0.18440715223505322
Score over time: 0.03
Time elapsed:  0.18318735130651476
Score over time: 0.393
Time elapsed:  0.6621520689291174
Score over time: 0.609
Time elapsed:  0.803173902776507
Score over time: 0.423
Time elapsed:  1.248896174802212
Score over time: 0.542
Time elapsed:  0.837688280198563
Score over time: 0.536
Time elapsed:  0.8174633539965726


In [25]:
for lr in [0.1, 0.5, 0.7, 0.8, 0.85, 0.9, 0.95, 0.99, 0.999, 1.0]:
    FrozenLakeQLearn_NoisyQ(iter_max=1000, learning_rate=lr)

Score over time: 0.183
Time elapsed:  0.6339902569307014
Score over time: 0.303
Time elapsed:  1.2211605430347845
Score over time: 0.552
Time elapsed:  1.2124137340579182
Score over time: 0.536
Time elapsed:  1.2240913429995999
Score over time: 0.088
Time elapsed:  2.419768098043278
Score over time: 0.009
Time elapsed:  2.8619415450375527
Score over time: 0.068
Time elapsed:  2.3281614179722965
Score over time: 0.019
Time elapsed:  3.0015432039508596
Score over time: 0.019
Time elapsed:  2.4791896189562976
Score over time: 0.032
Time elapsed:  0.30985289497766644


After running the Q-Learning algorithm for 1000 iterations for the following values... 
* learning rate = [0.1, 0.5, 0.7, 0.8, 0.85, 0.9, 0.95, 0.99, 0.999, 1.0]

... we find that the algorithm performs best with a (fixed) learning rate = 0.70 when gamma = 1.0.

In [27]:
for iterations in [1, 5, 10, 100, 1000, 10000, 100000]:
    FrozenLakeQLearn_NoisyQ(iter_max=iterations, g=1.0, learning_rate=0.7)

Score over time: 0.0
Time elapsed:  0.0007028409745544195
Score over time: 0.0
Time elapsed:  0.007706664968281984
Score over time: 0.0
Time elapsed:  0.010750383953563869
Score over time: 0.11
Time elapsed:  0.0879162399796769
Score over time: 0.552
Time elapsed:  1.3010737430304289
Score over time: 0.637
Time elapsed:  14.751405472983606
Score over time: 0.63671
Time elapsed:  141.6780458620051


Since Q-Learning is an online algorithm, it doesn't stop on a definitively optimal solution because it is blind to the MDP's true reward and transition functions. Instead, by running 100,000 iterations, the Q-Learning algorithm has accumulated an average score of 0.63119 by learning approximations of the "correct policy" from the history of the rewards it receives by making certain actions in each state. It is apparent that the training time increases proportionately to the number of iterations, but the asymptotic performance shows a learning curve that has plateaued. Running the algorithm for only 10,000 iterations results in an average score of 0.6256, not far from that of 100,000 iterations, but takes <9 seconds compared to the 91 seconds for 100,000 iterations.

Just as a sanity check, we can try Q-Learning using epsilon-greedy without decreasing epsilon over time. The performance *should* be worse than the above Q-Learning algorithm, where we began with a large epsilon (1.0) and asymptotically decreased it towards zero in order to prioritize exploration early on and exploit what the algorithm learned in later episodes. 

In [10]:
"""
Solving FrozenLake environment using Q-Learning
"""

def FrozenLakeQLearn_EpsilonGreedy(iter_max = 10000, learning_rate = 0.8, g = 0.95, epsilon = 0.1):
    # Hyperparameters
    num_episodes = iter_max
    # Set learning parameters
    lr = learning_rate
    y = g
    
    env.seed(0)
    np.random.seed(0)
    
    start = timer()
    
    #Initialize table with all zeros
    Q = np.zeros([env.observation_space.n,env.action_space.n])
    #create lists to contain total rewards and steps per episode
    #jList = []
    rList = []
    for i in range(num_episodes):
        #Reset environment and get first new observation
        s = env.reset()
        rAll = 0
        d = False
        j = 0
        #The Q-Table learning algorithm
        while j < 99:
            j+=1
            #Choose an action by greedily (with noise) picking from Q table
            if np.random.uniform(0, 1) < epsilon:
                a = np.random.choice(env.action_space.n)
            else:
                a = np.argmax(Q[s,:])
            #Get new state and reward from environment
            s1,r,d,_ = env.step(a)
            #Update Q-Table with new knowledge
            Q[s,a] = Q[s,a] + lr*(r + y*np.max(Q[s1,:]) - Q[s,a])
            rAll += r
            s = s1
            if d == True:
                break
        #jList.append(j)
        rList.append(rAll)
    
    end = timer()
    
    print ("Score over time: " +  str(sum(rList)/num_episodes))
    print('Time elapsed: ', end - start)

In [11]:
FrozenLakeQLearn_EpsilonGreedy()

Score over time: 0.1215
Time elapsed:  4.52724552503787


And indeed, we find that this overly naive approach to Q-Learning results in a lackluster performance of 0.1215 over 10,000 iterations. 

## Taxi

As mentioned, the Taxi Problem includes a larger state space and a longer time horizon between start and goal states, with steady punishment for taking longer, inefficient routes to passengers and destinations. We want to incentivize our algorithms to pursue the “long game”—that is, to see past the upfront penalties for taking lots of steps to eventually reach the +20 points for a successful dropoff—so we want a discount rate (gamma) close to 1.0. If we set gamma close to 0, the algorithm will instead be incentivized to make the agent wander around aimlessly before eventually entering an absorbing state for -10 points, which would be heavily discounted so as not to matter much at all.

### Value Iteration

In [2]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.
    """
    
    def one_step_lookahead(state, V):
        """
        Helper function to calculate the value for all action in a given state.
        
        Args:
            state: The state to consider (int)
            V: The value to use as an estimator, Vector of length env.nS
        
        Returns:
            A vector of length env.nA containing the expected value of each action.
        """
        A = np.zeros(env.nA)
        for a in range(env.nA):
            for prob, next_state, reward, done in env.P[state][a]:
                A[a] += prob * (reward + discount_factor * V[next_state])
        return A
    
    i = 0
    V = np.zeros(env.nS)
    while True:
        if i % 100 == 0:
            print('Step: ', i)
        i += 1
        # Stopping condition
        delta = 0
        # Update each state...
        for s in range(env.nS):
            # Do a one-step lookahead to find the best action
            A = one_step_lookahead(s, V)
            best_action_value = np.max(A)
            # Calculate delta across all states seen so far
            delta = max(delta, np.abs(best_action_value - V[s]))
            # Update the value function. Ref: Sutton book eq. 4.10. 
            V[s] = best_action_value        
        # Check if we can stop 
        if delta < theta:
            print ('Value-iteration converged at iteration# %d.' %(i+1))
            break
    
    # Create a deterministic policy using the optimal value function
    policy = np.zeros([env.nS, env.nA])
    for s in range(env.nS):
        # One step lookahead to find the best action for this state
        A = one_step_lookahead(s, V)
        best_action = np.argmax(A)
        # Always take the best action
        policy[s, best_action] = 1.0
    
    return policy, V

def evaluate_policy(env, policy, gamma = 1.0,  n = 100):
    """ Evaluates a policy by running it n times.
    returns:
    average total reward
    """
    steps = []
    scores = []
    for i in range(n):
        step, score = count(env, policy, gamma)
        steps.append(step)
        scores.append(score)
    return np.mean(steps), np.mean(scores)

# count the average number of steps taken to reach the goal state. 
def count(env, policy, gamma):
    curr_state = env.reset()
    counter = 0
    reward = None
    totalReward = 0
    while reward != 20:
        state, reward, done, info = env.step(np.argmax(policy[curr_state]))  
        totalReward += gamma ** counter * reward
        curr_state = state
        counter += 1
    return counter, totalReward

def TaxiVI(g = 0.99, n = 100):
    gamma = g

    env.seed(0)
    np.random.seed(0)

    start = timer()
    policy, V = value_iteration(env, discount_factor=gamma);
    end = timer()

    policy_steps, policy_score = evaluate_policy(env, policy, gamma, n=n)

    print('Policy average score = ', policy_score)
    print('Policy average steps = ', policy_steps)
    print('Time elapsed: ', end - start)
    

In [5]:
TaxiVI(g=0.99, n=100)

Step:  0
Step:  100
Step:  200
Step:  300
Step:  400
Step:  500
Step:  600
Step:  700
Step:  800
Step:  900
Step:  1000
Step:  1100
Step:  1200
Value-iteration converged at iteration# 1217.
Policy average score =  6.93708752167
Policy average steps =  12.5
Time elapsed:  11.448386250063777


For the sake of evaluating the average scores of the solution policy, we will increase the margin of error for convergence from 1e-20 to 1e-4 so that the algorithm will converge within a reasonable amount of time. When we run value iteration on gamma = 0.99, it converges in 1217 iterations and 11.79 seconds wall clock time. Evaluating the resultant policy 100 times gives an average 12.5 steps and score of 6.937. Without directly comparing to the results for PI, we can imagine that the solution policy generally takes the taxi through the optimal routes for each of the 20 subproblems, which take average 12.5 steps to complete, or about -13 points. That gives us an average score around 7 from successfully delivering the passenger to the destination.

### Policy Iteration

In [4]:
"""
Solving Taxi environment using Policy iteration.
"""

env_name  = 'Taxi-v2'
env = gym.make(env_name)
env = env.unwrapped

def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        delta = 0
        # For each state, perform a "full backup"
        for s in range(env.nS):
            v = 0
            # Look at the possible next actions
            for a, action_prob in enumerate(policy[s]):
                # For each action, look at the possible next states...
                for  prob, next_state, reward, done in env.P[s][a]:
                    # Calculate the expected value
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            # How much our value function changed (across any states)
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        # Stop evaluating once our value function change is below a threshold
        if delta < theta:
            break
    return np.array(V)

def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """

    def one_step_lookahead(state, V):
        """
        Helper function to calculate the value for all action in a given state.
        
        Args:
            state: The state to consider (int)
            V: The value to use as an estimator, Vector of length env.nS
        
        Returns:
            A vector of length env.nA containing the expected value of each action.
        """
        A = np.zeros(env.nA)
        for a in range(env.nA):
            for prob, next_state, reward, done in env.P[state][a]:
                A[a] += prob * (reward + discount_factor * V[next_state])
        return A
    
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    i = 0
    while True:
        print('Step: ', i)
        i += 1
        
        # Evaluate the current policy
        V = policy_eval_fn(policy, env, discount_factor)
        
        # Will be set to false if we make any changes to the policy
        policy_stable = True
        
        # For each state...
        for s in range(env.nS):
            # The best action we would take under the currect policy
            chosen_a = np.argmax(policy[s])
            
            # Find the best action by one-step lookahead
            # Ties are resolved arbitarily
            action_values = one_step_lookahead(s, V)
            best_a = np.argmax(action_values)
            
            # Greedily update the policy
            if chosen_a != best_a:
                policy_stable = False
            policy[s] = np.eye(env.nA)[best_a]
        
        # If the policy is stable we've found an optimal policy. Return it
        if policy_stable:
            print ('Policy-iteration converged at iteration# %d.' %(i+1))
            return policy, V
        
def evaluate_policy(env, policy, gamma = 1.0,  n = 100):
    """ Evaluates a policy by running it n times.
    returns:
    average total reward
    """
    steps = []
    scores = []
    for i in range(n):
        step, score = count(env, policy, gamma)
        steps.append(step)
        scores.append(score)
    return np.mean(steps), np.mean(scores)

# count the average number of steps taken to reach the goal state. 
def count(env, policy, gamma):
    curr_state = env.reset()
    counter = 0
    reward = None
    totalReward = 0
    while reward != 20:
        state, reward, done, info = env.step(np.argmax(policy[curr_state]))  
        totalReward += gamma ** counter * reward
        curr_state = state
        counter += 1
    return counter, totalReward

def TaxiPI(g = 0.99, n = 100):
    gamma = g

    env.seed(0)
    np.random.seed(0)

    start = timer()
    policy, V = policy_improvement(env, discount_factor=gamma);
    end = timer()

    policy_steps, policy_score = evaluate_policy(env, policy, gamma, n=n)

    print('Policy average score = ', policy_score)
    print('Policy average steps = ', policy_steps)
    print('Time elapsed: ', end - start)

In [60]:
TaxiPI(g=0.99, n=100)

Step:  0
Step:  1
Step:  2
Step:  3
Step:  4
Step:  5
Step:  6
Step:  7
Step:  8
Step:  9
Policy-iteration converged at iteration# 11.
Policy average score =  6.93708752167
Policy average steps =  12.5
Time elapsed:  67.718011562014


Of course, the solution policy from PI is the same, and so has the same average steps and average score as VI. However, policy iteration on gamma = 0.99 converges in 9 iterations, but 67.72 seconds wall clock time—over 5 times as long as VI! This is because policy evaluation implies solving the Bellman equation typically through a system of equations, which is very costly for large state-action sets. Although PI converges faster than VI in many scenarios including Frozen Lake, in this case, the additional computational complexity of the inner loop made PI converge slower than VI in real time. 

### Q-Learning
For the sake of comparison, we will use the same Q-Learning algorithm implementation as we did for Frozen Lake, including the fixed learning rate and exploration strategy of greedily choosing from the Q-table with noise that decays as iterations increase. This algorithm will have a Q-table of 500 (size of state space) × 6 (size of action space) = 3000. We choose to train for 10,000 iterations in order to hypothetically cover every state-action pair plenty of times when graphing performance against learning rate. Because our choice of gamma does not negatively affect time to convergence in Q-Learning, we will choose to use gamma = 1.0.

In [76]:
"""
Solving Taxi environment using Q-Learning
"""

env_name = 'Taxi-v2'
env = gym.make(env_name)
env = env.unwrapped

def TaxiQLearn_NoisyQ(iter_max = 100, t_max = 150, learning_rate = 0.8, g = 1.0):
    # Hyperparameters
    num_episodes = iter_max
    # Set learning parameters
    lr = learning_rate
    y = g
    
    env.seed(0)
    np.random.seed(0)
    
    start = timer()
    
    #Initialize table with all zeros
    Q = np.zeros([env.observation_space.n,env.action_space.n])
    #create lists to contain total rewards and steps per episode
    jList = []
    rList = []
    for i in range(num_episodes):
        #Reset environment and get first new observation
        s = env.reset()
        rAll = 0
        d = False
        j = 0
        #The Q-Table learning algorithm
        while j < t_max:
            j+=1
            #Choose an action by greedily (with noise) picking from Q table
            a = np.argmax(Q[s,:] + np.random.randn(1,env.action_space.n)*(1./(i+1)))
            #Get new state and reward from environment
            s1,r,d,_ = env.step(a)
            #Update Q-Table with new knowledge
            Q[s,a] = Q[s,a] + lr*(r + y*np.max(Q[s1,:]) - Q[s,a])
            rAll += r
            s = s1
            if d == True:
                break
        jList.append(j)
        rList.append(rAll)
    
    end = timer()
    
    print ("Score over time: " +  str(sum(rList)/num_episodes))
    print ("Average steps: " +  str(sum(jList)/num_episodes))
    print('Time elapsed: ', end - start)

In [77]:
for lr in [0.6, 0.7, 0.8, 0.9, 0.99, 1.0]:
    TaxiQLearn_NoisyQ(iter_max=10000, t_max=300, learning_rate=lr)

Score over time: 5.4705
Average steps: 14.3946
Time elapsed:  5.054019118892029
Score over time: 5.6538
Average steps: 14.256
Time elapsed:  4.794186649029143
Score over time: 5.7583
Average steps: 14.1917
Time elapsed:  5.51989335892722
Score over time: 5.8352
Average steps: 14.1736
Time elapsed:  5.051946762949228
Score over time: 5.9602
Average steps: 14.1137
Time elapsed:  5.379903438966721
Score over time: 5.8457
Average steps: 14.1196
Time elapsed:  5.5309618200408295


The average steps for each learning rate are all around 14, but the average reward per episode peaks sharply at learning rate = 0.99. Graphing the performance, we see that the average scores of 10,000 or 100,000 cumulative episodes are between 5.9 and 8.1, asymptotically approaching 12.5. This is expected behavior, and closely mirrors that seen in Q-Learning for Frozen Lake, since the RL algorithm can only approximate the optimal policy based on the values populated in the Q-table by the episodes it has trained on but runs into linear training time constraints. 

This phenomenon of Q-Learning approaching but not reaching the optimal policy found by VI and PI can also be seen in the average number of steps for each iteration:

In [78]:
for iterations in [1, 5, 10, 100, 200, 500, 1000, 10000, 100000]:
    TaxiQLearn_NoisyQ(iter_max=iterations, g=1.0, learning_rate=0.99)

Score over time: -438.0
Average steps: 150.0
Time elapsed:  0.012774679926224053
Score over time: -472.2
Average steps: 150.0
Time elapsed:  0.07571056205779314
Score over time: -424.5
Average steps: 150.0
Time elapsed:  0.06688580999616534
Score over time: -170.03
Average steps: 111.08
Time elapsed:  0.4407082989346236
Score over time: -96.825
Average steps: 75.45
Time elapsed:  0.5444652970181778
Score over time: -39.042
Average steps: 40.194
Time elapsed:  0.7720109439687803
Score over time: -15.84
Average steps: 26.655
Time elapsed:  0.9311205690028146
Score over time: 5.8963
Average steps: 14.0852
Time elapsed:  5.377058368991129
Score over time: 8.09169
Average steps: 12.80646
Time elapsed:  46.427650494966656


In [83]:
"""
Solving Taxi environment using Q-Learning
"""

env_name = 'Taxi-v2'
env = gym.make(env_name)
env = env.unwrapped

def TaxiQLearn_EpsilonGreedy(iter_max = 100, t_max = 150, learning_rate = 0.8, g = 1.0, epsilon = 0.05):
    # Hyperparameters
    num_episodes = iter_max
    # Set learning parameters
    lr = learning_rate
    y = g
    
    env.seed(0)
    np.random.seed(0)
    
    start = timer()
    
    #Initialize table with all zeros
    Q = np.zeros([env.observation_space.n,env.action_space.n])
    #create lists to contain total rewards and steps per episode
    jList = []
    rList = []
    for i in range(num_episodes):
        #Reset environment and get first new observation
        s = env.reset()
        rAll = 0
        d = False
        j = 0
        #The Q-Table learning algorithm
        while j < t_max:
            j+=1
            #Choose an action by greedily (with noise) picking from Q table
            if np.random.uniform(0, 1) < epsilon:
                a = np.random.choice(env.action_space.n)
            else:
                a = np.argmax(Q[s,:])
            #Get new state and reward from environment
            s1,r,d,_ = env.step(a)
            #Update Q-Table with new knowledge
            Q[s,a] = Q[s,a] + lr*(r + y*np.max(Q[s1,:]) - Q[s,a])
            rAll += r
            s = s1
            if d == True:
                break
        jList.append(j)
        rList.append(rAll)
    
    end = timer()
    
    print ("Score over time: " +  str(sum(rList)/num_episodes))
    print ("Average steps: " +  str(sum(jList)/num_episodes))
    print('Time elapsed: ', end - start)

When Q-Learning runs for 100,000 episodes, the cumulative number of steps averages to 12.81 per episode, quite close to the 12.5 steps in VI and PI. Finally, if we contrast with a simple epsilon-greedy approach to selecting actions in the Q-Learning algorithm where epsilon stays fixed at 0.05, we see that the performance is worse for the same reasons as in Frozen Lake—the algorithm is unable to take advantage of its training time to more fully explore the state-action space early on, nor is it able to fully exploit the information it has gained later on. It results in an average score of only 4.914, contrasted with 8.092 seen in the above greedy noisy Q-Learning algorithm. 

In [84]:
TaxiQLearn_EpsilonGreedy(iter_max=100000, g=1.0, learning_rate=0.99)

Score over time: 4.91436
Average steps: 14.10828
Time elapsed:  35.80320289905649
